In [1]:
from sqlalchemy import create_engine
import pandas as pd
import datetime as dt
import json

In [2]:
# Connect to database
database_url = 'postgresql://postgres:"your PostgreSQL password"@localhost:5432/online_shopping'
engine = create_engine(database_url)


In [3]:
# Query the transaction table
query = 'SELECT * FROM transaction'
transaction_df = pd.read_sql(query, engine)
transaction_df.head()


,id,transaction_id,customer_id,transaction_date,product_id,quantity,delivery_charges,coupon_status,coupon_code,discount_pct,gst,month
0,0.0,16679.0,17850.0,2019-01-01,GGOENEBJ079499,1.0,6.5,Used,ELEC10,10.0,0.1,1.0
1,1.0,16680.0,17850.0,2019-01-01,GGOENEBJ079499,1.0,6.5,Used,ELEC10,10.0,0.1,1.0
2,2.0,16696.0,17850.0,2019-01-01,GGOENEBQ078999,2.0,6.5,Not Used,ELEC10,10.0,0.1,1.0
3,3.0,16699.0,17850.0,2019-01-01,GGOENEBQ079099,1.0,6.5,Clicked,ELEC10,10.0,0.1,1.0
4,4.0,16700.0,17850.0,2019-01-01,GGOENEBJ079499,1.0,6.5,Clicked,ELEC10,10.0,0.1,1.0


In [4]:
# Convert data type to datetime
transaction_df["transaction_date"] = pd.to_datetime(transaction_df["transaction_date"])

In [5]:
# Query the product table
query2 = 'SELECT * FROM product'
product_df = pd.read_sql(query2, engine)
product_df.head()

,product_id,product_description,product_category,avg_price
0,GGOENEBJ079499,Nest Learning Thermostat 3rd Gen-USA - Stainle...,Nest-USA,153.71
1,GGOENEBQ078999,Nest Cam Outdoor Security Camera - USA,Nest-USA,122.77
2,GGOENEBQ079099,Nest Protect Smoke + CO White Battery Alarm-USA,Nest-USA,81.50
3,GGOENEBB078899,Nest Cam Indoor Security Camera - USA,Nest-USA,122.77
4,GGOENEBQ079199,Nest Protect Smoke + CO White Wired Alarm-USA,Nest-USA,81.50


In [6]:
# Merge the product and transaction dataframe
new_transaction_df=pd.merge(product_df, transaction_df, on=["product_id"])

#Adding total sales revenue column
new_transaction_df["total_sales_revenue"] = new_transaction_df["avg_price"] * new_transaction_df["quantity"]

# Update the 'total_sales_revenue' column
new_transaction_df.loc[new_transaction_df["coupon_status"] == "Used", "total_sales_revenue"] = (
    (new_transaction_df["avg_price"] - (new_transaction_df["discount_pct"] * 0.01 * new_transaction_df["avg_price"])) 
    * new_transaction_df["quantity"]
)

# Display the dataframe
new_transaction_df.head()

,product_id,product_description,product_category,avg_price,id,transaction_id,customer_id,transaction_date,quantity,delivery_charges,coupon_status,coupon_code,discount_pct,gst,month,total_sales_revenue
0,GGOENEBJ079499,Nest Learning Thermostat 3rd Gen-USA - Stainle...,Nest-USA,153.71,0.0,16679.0,17850.0,2019-01-01,1.0,6.5,Used,ELEC10,10.0,0.1,1.0,138.339
1,GGOENEBJ079499,Nest Learning Thermostat 3rd Gen-USA - Stainle...,Nest-USA,153.71,1.0,16680.0,17850.0,2019-01-01,1.0,6.5,Used,ELEC10,10.0,0.1,1.0,138.339
2,GGOENEBJ079499,Nest Learning Thermostat 3rd Gen-USA - Stainle...,Nest-USA,153.71,4.0,16700.0,17850.0,2019-01-01,1.0,6.5,Clicked,ELEC10,10.0,0.1,1.0,153.710
3,GGOENEBJ079499,Nest Learning Thermostat 3rd Gen-USA - Stainle...,Nest-USA,153.71,5.0,16701.0,17850.0,2019-01-01,1.0,6.5,Clicked,ELEC10,10.0,0.1,1.0,153.710
4,GGOENEBJ079499,Nest Learning Thermostat 3rd Gen-USA - Stainle...,Nest-USA,153.71,6.0,16702.0,17850.0,2019-01-01,2.0,6.5,Clicked,ELEC10,10.0,0.1,1.0,307.420


In [7]:
# Calculate monthly total sales revenue
# Group by month and calculate total sales

monthly_sales = new_transaction_df.groupby(new_transaction_df['transaction_date'].dt.to_period('M'))['total_sales_revenue'].sum()
monthly_sales.head()


transaction_date
2019-01    386203.774
2019-02    290555.016
2019-03    319208.516
2019-04    393338.798
2019-05    299577.512
Freq: M, Name: total_sales_revenue, dtype: float64

In [8]:
# Create a new dataframe with transaction_date, product_category and total_sales_revenue as columns
sales = new_transaction_df[["transaction_date", "product_category", "total_sales_revenue"]]
sales.head()

,transaction_date,product_category,total_sales_revenue
0,2019-01-01,Nest-USA,138.339
1,2019-01-01,Nest-USA,138.339
2,2019-01-01,Nest-USA,153.710
3,2019-01-01,Nest-USA,153.710
4,2019-01-01,Nest-USA,307.420


In [9]:
# Create a copy of the dataframe
sales_new = sales.copy()
# Add year_month column and extract the month and year for grouping
sales_new["year_month"] = sales_new["transaction_date"].dt.to_period("M")
sales_new.head()

,transaction_date,product_category,total_sales_revenue,year_month
0,2019-01-01,Nest-USA,138.339,2019-01
1,2019-01-01,Nest-USA,138.339,2019-01
2,2019-01-01,Nest-USA,153.710,2019-01
3,2019-01-01,Nest-USA,153.710,2019-01
4,2019-01-01,Nest-USA,307.420,2019-01


In [10]:
# Group by year_month and product_category, then calculate sum of the total_sales_revenue
grouped_sales = sales_new.groupby(["year_month", "product_category"])["total_sales_revenue"].sum().reset_index()
grouped_sales.head()

,year_month,product_category,total_sales_revenue
0,2019-01,Accessories,34.990
1,2019-01,Android,70.116
2,2019-01,Apparel,36970.926
3,2019-01,Bags,10872.463
4,2019-01,Bottles,551.450


In [11]:
# Step 4: Pivot data for nested dictionary format for the javascript file
sales_json = grouped_sales.pivot(index="year_month", columns="product_category", values="total_sales_revenue").fillna(0)
sales_json.head()

product_category,Accessories,Android,Apparel,Bags,Bottles,Drinkware,Gift Cards,Headgear,Housewares,Lifestyle,Nest,Nest-Canada,Nest-USA,Notebooks & Journals,Office,Waze
year_month,,,,,,,,,,,,,,,,
2019-01,34.990,70.116,36970.926,10872.463,551.450,14421.308,263.058,2070.111,787.950,5512.440,0.0,9311.436,271770.640,6133.506,26997.612,435.768
2019-02,0.000,20.620,34586.044,17613.130,299.572,13231.992,386.850,4132.466,74.984,6164.838,0.0,3764.430,187047.846,4128.636,18875.630,227.978
2019-03,33.590,107.140,54553.202,13052.369,421.918,19769.645,368.296,3052.022,415.502,6233.848,0.0,7109.585,183654.006,4134.218,25511.713,791.462
2019-04,113.921,35.470,86241.059,12171.215,459.980,26596.640,711.110,10457.939,76.014,8772.844,0.0,5582.035,181302.211,23879.903,36283.974,654.483
2019-05,105.844,84.240,59917.264,9919.378,928.618,16237.094,1330.764,3090.728,421.064,6471.904,0.0,3741.712,167290.626,6067.474,23455.704,515.098


In [12]:
# Convert the DataFrame 'sales_json' to a JSON with an index-based orientation 
result_json = sales_json.to_json(orient ='index')
# Display the JSON string
result_json

'{"2019-01":{"Accessories":34.99,"Android":70.116,"Apparel":36970.926,"Bags":10872.463,"Bottles":551.45,"Drinkware":14421.308,"Gift Cards":263.058,"Headgear":2070.111,"Housewares":787.95,"Lifestyle":5512.44,"Nest":0.0,"Nest-Canada":9311.436,"Nest-USA":271770.64,"Notebooks & Journals":6133.506,"Office":26997.612,"Waze":435.768},"2019-02":{"Accessories":0.0,"Android":20.62,"Apparel":34586.044,"Bags":17613.13,"Bottles":299.572,"Drinkware":13231.992,"Gift Cards":386.85,"Headgear":4132.466,"Housewares":74.984,"Lifestyle":6164.838,"Nest":0.0,"Nest-Canada":3764.43,"Nest-USA":187047.846,"Notebooks & Journals":4128.636,"Office":18875.63,"Waze":227.978},"2019-03":{"Accessories":33.59,"Android":107.14,"Apparel":54553.202,"Bags":13052.369,"Bottles":421.918,"Drinkware":19769.645,"Gift Cards":368.296,"Headgear":3052.022,"Housewares":415.502,"Lifestyle":6233.848,"Nest":0.0,"Nest-Canada":7109.585,"Nest-USA":183654.006,"Notebooks & Journals":4134.218,"Office":25511.713,"Waze":791.462},"2019-04":{"Acces

In [13]:
# Prepare JavaScript  content
js_content = f"let salesData = {result_json};"

# Save to a file
with open("data_new.js", "w") as file:
    file.write(js_content)

In [14]:
# Close engine
engine.dispose()